# Data load

In [2]:
import pandas as pd

data = pd.read_csv('athlete_events.csv')
data.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [20]:
data.dtypes

ID          int64
Name       object
Sex        object
Age       float64
Height    float64
Weight    float64
Team       object
NOC        object
Games      object
Year        int64
Season     object
City       object
Sport      object
Event      object
Medal      object
dtype: object

# Clean up

In [3]:
# drop unnecessary columns
ol_data = data.drop(['ID', 'Games'], axis=1)
ol_data.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Year,Season,City,Sport,Event,Medal
0,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


# Analysis

## Valores faltantes

In [19]:
# Columnas cualitativas de interés
cols_cualitativas = ["Name", "Sex", "Team", "NOC", "Season", "City", "Sport", "Event", "Medal"]

# Crear un DataFrame vacío para almacenar las estadísticas
estadisticas = pd.DataFrame(
    columns=[
        "Columna",
        "Moda",
        "Moda (#)",
        "Moda (%)",
        "Valores unicos (#)",
        "Valores unicos (%)",
        "Valores nulos (%)",
        # "Porcentaje_Errores",
    ]
)

for col in cols_cualitativas:
    # Moda
    moda = data[col].mode()[0]
    
    # Cantidad de veces que se repite la moda
    moda_count = data[col].value_counts().iloc[0]

    # Porcentaje de la moda
    moda_porcentaje = moda_count / len(data) * 100

    # Cantidad de valores diferentes
    valores_unicos = data[col].nunique()

    # Porcentaje de cada tipo de valor
    valor_porcentajes = data[col].nunique() / len(data) * 100

    # Porcentaje de datos faltantes
    porcentaje_nulos = data[col].isna().mean() * 100

    # Porcentaje de datos erróneos (debes definir qué valores se consideran erróneos)
    # valores_errores = ["NaN"]  # Ejemplo de valores erróneos
    # mascara_errores = data[col].isin(valores_errores)
    # porcentaje_errores = mascara_errores.mean() * 100

    # Agregar una fila al DataFrame de estadísticas
    nueva_fila = pd.DataFrame(
        {
            "Columna": [col],
            "Moda": [moda],
            "Moda (#)": [moda_count],
            "Moda (%)": [moda_porcentaje],
            "Valores unicos (#)": [valores_unicos],
            "Valores unicos (%)": [valor_porcentajes],
            "Valores nulos (%)": [porcentaje_nulos],
            # "Porcentaje_Errores": [porcentaje_errores],
        }
    )
    estadisticas = pd.concat([estadisticas, nueva_fila], ignore_index=True)
    
    # Guardar datos en un archivo CSV
    estadisticas.to_csv("descriptors/olympics_quality.csv", index=False, decimal=",")

# Mostrar el DataFrame de estadísticas
estadisticas


C:\Users\alvar\AppData\Local\Temp\ipykernel_3768\3974041509.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estadisticas = pd.concat([estadisticas, nueva_fila], ignore_index=True)


,Columna,Moda,Moda (#),Moda (%),Valores unicos (#),Valores unicos (%),Valores nulos (%)
0,Name,Robert Tait McKenzie,58,0.021393,134732,49.695333,0.000000
1,Sex,M,196594,72.512873,2,0.000738,0.000000
2,Team,United States,17847,6.582791,1184,0.436713,0.000000
3,NOC,USA,18853,6.953850,230,0.084835,0.000000
4,Season,Summer,222552,82.087372,2,0.000738,0.000000
5,City,London,22426,8.271736,42,0.015492,0.000000
6,Sport,Athletics,38624,14.246300,66,0.024344,0.000000
7,Event,Football Men's Football,5733,2.114593,765,0.282167,0.000000
8,Medal,Gold,13372,4.932206,3,0.001107,85.326207


In [10]:
# count the number of times Robert Tait McKenzie appears in the data
robert = data[data['Name'] == 'Robert Tait McKenzie']
len(robert)

58

In [80]:
# count the number of missing values in each column
missing = ol_data.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.drop('Medal', inplace=True)
missing

Age        9474
Height    60171
Weight    62875
dtype: int64

In [15]:
# count the rows with two values in the team column
teams = ol_data['Team'].value_counts()
teams = teams[teams == 2]
teams

Team
Starlight III             2
Pirouette-5               2
Zefyros                   2
France/Great Britain      2
Portugal-2                2
                         ..
North Borneo              2
Ravenswood Boat Club-2    2
Bohemia-1                 2
Gem II                    2
Esterel-1                 2
Name: count, Length: 265, dtype: int64

In [14]:
# get the rows with missing age values
missing_age = ol_data[ol_data['Age'].isnull()]
missing_age.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Year,Season,City,Sport,Event,Medal
147,Mohamed Jamshid Abadi,M,NaN,NaN,NaN,Iran,IRI,1948,Summer,London,Boxing,Boxing Men's Heavyweight,NaN
152,Georgi Abadzhiev,M,NaN,NaN,NaN,Bulgaria,BUL,1924,Summer,Paris,Cycling,"Cycling Men's Road Race, Individual",NaN
153,Georgi Abadzhiev,M,NaN,NaN,NaN,Bulgaria,BUL,1924,Summer,Paris,Cycling,"Cycling Men's Road Race, Team",NaN
162,Mohamed Abakkar,M,NaN,156.0,48.0,Sudan,SUD,1972,Summer,Munich,Boxing,Boxing Men's Flyweight,NaN
212,Sayed Fahmy Abaza,M,NaN,NaN,NaN,Egypt,EGY,1920,Summer,Antwerpen,Football,Football Men's Football,NaN


In [9]:
# show the rows with missing values
missing_rows = ol_data[ol_data.isnull().any(axis=1)]
missing_rows.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Year,Season,City,Sport,Event,Medal
0,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


## `.describe` con valores de skewness y curtosis

In [7]:
import numpy as np

# Calculate skewness for numeric columns only
numeric_cols = ol_data.select_dtypes(include=[np.number])
skewness = pd.DataFrame(numeric_cols.skew(), columns=['skewness']).T

# Calculate kurtosis for numeric columns only
kurtosis = pd.DataFrame(numeric_cols.kurt(), columns=['kurtosis']).T

# Calculate IQR for numeric columns only
iqr = numeric_cols.apply(lambda x: x.quantile(0.75) - x.quantile(0.25))
iqr = pd.DataFrame(iqr, columns=['IQR']).T

# Get the description for all columns
desc = ol_data.describe()

# Append skewness and kurtosis to the description
desc = pd.concat([desc, skewness, kurtosis, iqr])

# Export the DataFrame to a CSV file
desc.to_csv('descriptors/olympic.csv', decimal=',')

print(desc)

                    Age         Height         Weight           Year
count     261642.000000  210945.000000  208241.000000  271116.000000
mean          25.556898     175.338970      70.702393    1978.378480
std            6.393561      10.518462      14.348020      29.877632
min           10.000000     127.000000      25.000000    1896.000000
25%           21.000000     168.000000      60.000000    1960.000000
50%           24.000000     175.000000      70.000000    1988.000000
75%           28.000000     183.000000      79.000000    2002.000000
max           97.000000     226.000000     214.000000    2016.000000
skewness       1.747123       0.018477       0.797169      -0.817736
kurtosis       6.270642       0.177728       2.017523      -0.206948
IQR            7.000000      15.000000      19.000000      42.000000


## Entradas por temporada

In [46]:
# count the number of rows with each season
seasons = ol_data['Season'].value_counts()
seasons

Season
Summer    222552
Winter     48564
Name: count, dtype: int64

## Ciudades que han hosteado las olimpiadas

In [47]:
# count the number of cities in which the olympics were held
cities = ol_data['City'].nunique()
cities

42

## Veces que se han hosteado las olimpiadas en cada ciudad

In [49]:
# count how many times the olympics were held in each city
city_counts = ol_data['City'].value_counts()
city_counts.head()

City
London            22426
Athina            15556
Sydney            13821
Atlanta           13780
Rio de Janeiro    13688
Name: count, dtype: int64

## Atletas por pais

In [51]:
# count the number of athletes from each country
athletes = ol_data['NOC'].value_counts()
athletes.head()

NOC
USA    18853
FRA    12758
GBR    12256
ITA    10715
GER     9830
Name: count, dtype: int64

## Atletas por temporada

In [55]:
# count the number of athletes that are from each season
athletes_season = ol_data.groupby('Season')['Name'].nunique()
athletes_season

Season
Summer    116122
Winter     18923
Name: Name, dtype: int64

## Atletas por año

In [57]:
# count the number of athletes per year
athletes_year = ol_data.groupby('Year')['Name'].nunique()
athletes_year.head()

Year
1896     176
1900    1220
1904     650
1906     841
1908    2024
Name: Name, dtype: int64

## Medallas ganadas por pais

In [66]:
# Count the number of medals won by each country
medal_counts = ol_data.groupby('NOC')['Medal'].count()
medal_counts.sort_values(ascending=False).head()

NOC
USA    5637
URS    2503
GER    2165
GBR    2068
FRA    1777
Name: Medal, dtype: int64

## Medallas ganadas por atleta

In [67]:
# count the atheletes who won the most medals
most_medals = ol_data.groupby('Name')['Medal'].count()
most_medals.sort_values(ascending=False).head()

Name
Michael Fred Phelps, II               28
Larysa Semenivna Latynina (Diriy-)    18
Nikolay Yefimovich Andrianov          15
Takashi Ono                           13
Borys Anfiyanovych Shakhlin           13
Name: Medal, dtype: int64

## Atletas por deporte

In [68]:
# count the most popular sports == the sports with the most athletes
popular_sports = ol_data['Sport'].value_counts()
popular_sports.head()

Sport
Athletics     38624
Gymnastics    26707
Swimming      23195
Shooting      11448
Cycling       10859
Name: count, dtype: int64

## Atletas por evento

In [69]:
# count the most popular events == the events with the most athletes
popular_events = ol_data['Event'].value_counts()
popular_events.head()

Event
Football Men's Football        5733
Ice Hockey Men's Ice Hockey    4762
Hockey Men's Hockey            3958
Water Polo Men's Water Polo    3358
Basketball Men's Basketball    3280
Name: count, dtype: int64

## Deportes antiguos que aun se existen

In [82]:
# Find the first year each event was held
first_year = ol_data.groupby('Event')['Year'].min()

# Find the most recent year in the data
latest_year = ol_data['Year'].max()

# Find the events held in the most recent year
latest_events = ol_data[ol_data['Year'] == latest_year]['Event'].unique()

# Count the number of events that were first held in the earliest year and are still being held
oldest_events_still_held = first_year[first_year.index.isin(latest_events)]

oldest_events_still_held

Event
Archery Men's Individual                          1972
Archery Men's Team                                1988
Archery Women's Individual                        1972
Archery Women's Team                              1988
Athletics Men's 1,500 metres                      1896
                                                  ... 
Wrestling Women's Flyweight, Freestyle            2004
Wrestling Women's Heavyweight, Freestyle          2004
Wrestling Women's Light-Heavyweight, Freestyle    2016
Wrestling Women's Lightweight, Freestyle          2004
Wrestling Women's Middleweight, Freestyle         2004
Name: Year, Length: 306, dtype: int64

## Atletas por sexo

In [83]:
# count the number of athletes for each sex
athletes_sex = data['Sex'].value_counts()
athletes_sex

Sex
M    196594
F     74522
Name: count, dtype: int64

## Atletas por sexo y pais

In [73]:
# count the athletes by sex and country
athletes_by_sex_country = data.groupby(['Sex', 'NOC']).size().reset_index(name='Count')
athletes_by_sex_country.head()


,Sex,NOC,Count
0,F,AFG,5
1,F,AHO,12
2,F,ALB,27
3,F,ALG,94
4,F,AND,42


## Atletas por sexo y deporte

In [75]:
# count the athletes by sex and sport
athletes_by_sex_sport = data.groupby(['Sex', 'Sport']).size().reset_index(name='Count')
athletes_by_sex_sport.head()

,Sex,Sport,Count
0,F,Alpine Skiing,3398
1,F,Alpinism,1
2,F,Archery,1015
3,F,Art Competitions,377
4,F,Athletics,11666


## Medallas ganadas por pais (por tipo de medalla)

In [76]:
# count the countries that have won the most of each medal
gold_countries = data[data['Medal'] == 'Gold']['NOC'].value_counts()
silver_countries = data[data['Medal'] == 'Silver']['NOC'].value_counts()
bronze_countries = data[data['Medal'] == 'Bronze']['NOC'].value_counts()

gold_countries.head(), silver_countries.head(), bronze_countries.head()

(NOC
 USA    2638
 URS    1082
 GER     745
 GBR     678
 ITA     575
 Name: count, dtype: int64,
 NOC
 USA    1641
 GBR     739
 URS     732
 GER     674
 FRA     610
 Name: count, dtype: int64,
 NOC
 USA    1358
 GER     746
 URS     689
 FRA     666
 GBR     651
 Name: count, dtype: int64)

## Deportes por temporada

In [78]:
# count the number of sports held in each season
sports_season = data.groupby('Season')['Sport'].nunique()
sports_season

Season
Summer    52
Winter    17
Name: Sport, dtype: int64

## Eventos por temporada

In [79]:
# count the number of events held in each season
events_season = data.groupby('Season')['Event'].nunique()
events_season

Season
Summer    651
Winter    119
Name: Event, dtype: int64